In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score
from time import strptime,time,mktime
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso,Ridge,LinearRegression
from sklearn.decomposition import PCA
from seaborn import heatmap,pairplot
from sklearn.feature_selection import SelectKBest,mutual_info_regression

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
colors=['red','green','blue','yellow','violet','#00ffff','#ff00bf','#937f3c','#937f91','#630ff0']

In [ ]:
df1=pd.read_csv("../resources/Excel_sheets/Car_sales.csv")
df1.head()

In [ ]:
df1.info()

lowercasing column names for easiness

In [ ]:
df1.columns=[x.lower() for x in df1.columns]
df1.head(2)

## DATA CLEANING

In [ ]:
df1.isnull().sum()

In [ ]:
plt.scatter(df1.__year_resale_value,df1.price_in_thousands,color=colors[4])
plt.xlabel('__year_resale_value')
plt.ylabel('price')
plt.show()

from this scatter plot it is evident that price and year_resale_value are linearly related
hence we can use linear_regression model to predict the missing values

In [ ]:
len(df1[df1[['__year_resale_value','price_in_thousands']].isna().all(axis=1)])

both are not null hence we can use linear_regression model with any problem

In [ ]:
d_df=df1.dropna(subset=['__year_resale_value','price_in_thousands'])
y=d_df.__year_resale_value.values
X=d_df.price_in_thousands.values.reshape(-1,1)

In [ ]:
model1=LinearRegression()
model1.fit(X,y)
model1.score(X,y)

In [ ]:
df2=df1.copy()
d=df1[df1.__year_resale_value.isna()]
n=d.index
df2.loc[n,['__year_resale_value']]=model1.predict(d[['price_in_thousands']])
df2.__year_resale_value.isna().sum()

In [ ]:
m=model1.coef_
b=model1.intercept_

In [ ]:
d=df2[df2.price_in_thousands.isna()]
n=d.index
df2.loc[n,['price_in_thousands']]=(d.__year_resale_value.values-b)/m[0]
df2.price_in_thousands.isna().sum()

In [ ]:
plt.scatter(df2.price_in_thousands,df2.__year_resale_value,color=colors[7])
plt.plot(df2.price_in_thousands,model1.predict(df2[['price_in_thousands']]),color=colors[0])
plt.xlabel('__year_resale_value')
plt.ylabel('price')
plt.show()

filling null in other columns with  mean

In [ ]:
df3=df2.copy()
for column in df2.columns:
    if df2[column].isnull().sum():
        df3[column]=df2[column].fillna(df2[column].mean())
        
df4.isnull().sum()

### Feature Engineering

In [ ]:
df3.latest_launch.unique()

All the launch dates are within 2012 ,

i will create a new column which coverts date into no of days from launch date to 1/1/13

In [ ]:
t=mktime(strptime('1/1/2013',"%m/%d/%Y"))

In [ ]:
df4=df3.copy()
day=24*3600
df4['days_after_launch(1/1/13)']=df3['latest_launch'].apply(lambda x: int((t-mktime(strptime(x,"%m/%d/%Y")))/day))
df4.drop('latest_launch',axis=1,inplace=True)
df4.head(2)

moving target variable to left for easiness

In [ ]:
cols=df4.columns.tolist()
cols.append(cols.pop(cols.index('price_in_thousands')))
df4=df4[cols]
df4.head(2)

### Outlier detection

In [ ]:
df4.describe()

In [ ]:
figu,ax=plt.subplots(1,2,figsize=(10,4))
ax[0].hist(df4.sales_in_thousands)
ax[1].hist(df4.__year_resale_value)
ax[0].set_ylabel('count')
ax[1].set_ylabel('count')
ax[0].set_xlabel('sales_in_thousands')
ax[1].set_xlabel('year_resale_value')
plt.show()

i will use 3 std as criteria for outlier deleting all which are more than 3 std

In [ ]:
def remove_outlier(df):
    num_df_cols=df.select_dtypes(include=[np.number]).columns
    d=pd.DataFrame(columns=df.columns.to_list().append('outlier'))
    for col in num_df_cols:
        mean=df[col].mean()
        std=df[col].std()
        u_limit=mean+3*std
        l_limit=mean-3*std
        out=df[(df[col]<l_limit) | (df[col]>u_limit)].copy()
        out['outlier']=col
        d=pd.concat([out,d])
        df=df[(df[col]>l_limit) & (df[col]<u_limit)]
    return (df,d)
    
df5,d=remove_outlier(df4)
d

most of the outliers are seems to be valid but still it is better to remove 
them for building better model

In [ ]:
df4.shape[0]-df5.shape[0]

we removed 21 outliers

##  Data Analysis & Visualisation

1. Manufacturer

In [ ]:
df4['manufacturer'].nunique()

In [ ]:
Manf_name=df4['manufacturer'].unique()
Manf_name

#### Top 10 companies with highest no of cars[models]

In [ ]:
high_mod=df4['manufacturer'].value_counts()[:10]
plt.figure(figsize=(11,4))
plt.bar(high_mod.index.values,high_mod.values,color='#937f91')
plt.ylabel('No of Models')
plt.xlabel('Manufacturer')
plt.show()

Dodge and Toyota has highest no of model

2. Model

In [ ]:
d=df4['model'].value_counts()
mod=d[d>1].index.values
mod

In [ ]:
df4[df4.model==mod[0]].manufacturer.values

Doged and Polymouth has cars of same model name as Neon

3. Sales

In [ ]:
sales=df4.groupby('manufacturer')['sales_in_thousands'].sum().sort_values(ascending =False)
sal_10=sales[:10]
l=sal_10.index[:9].to_list()
l.append('other')
sal_10.index=l
sal_10[9]=sum(sales.values[10:])

In [ ]:
plt.figure(figsize=(7,7))
plt.pie(sal_10,startangle=40,autopct='%1.1f%%',shadow=True,labels=sal.index)
plt.show()

ford has highest sales 

In [ ]:
df6=df4.copy()
df6['sell_price']=df6.sales_in_thousands*df6.price_in_thousands
sell_price=df6.groupby('manufacturer')['sell_price'].sum().sort_values(ascending =False)[:10]

In [ ]:
figu,ax=plt.subplots(figsize=(15,7))
ax.barh(sell_price.index.values,sell_price.values*1000,color='#337091')
ax.invert_yaxis()
for i in ax.patches:
    plt.text(i.get_width()+0.1,i.get_y()+0.5,str(round(i.get_width(),2)),fontsize=12,fontweight="bold",color="grey")
ax.set_title("10 companies generated highest profit",loc="right")
plt.show()

Ford is generating highest revenue by providing more no of models

In [ ]:
df6.head()

In [ ]:
pairplot(df4[['sales_in_thousands','horsepower','price_in_thousands','fuel_efficiency']])

* we can observe all the histograms are right skewed and some ouliers present in all graph
(outliers are already deleted here i used old dataframe)
* cars with lower price sell more than higher one 
* cars with higher horsepower has lesser fuelefficiency



## DATA POCESSING AND BUILDING MODEL FOR TARGET AS PRICE

### DATA PREPROCESSING

In [ ]:
df5[df5.model=='Neon']

model named neon has 2 manufacturers and we can observe that two model has same features
except sales and lauch date this probably be data error we shall delete these

In [ ]:
df7=df5.copy()

In [ ]:
df7.drop(df7[df7.model=='Neon'].index,inplace=True)

In [ ]:
nam=df7[['model','price_in_thousands']].sort_values(by='price_in_thousands').model.values
encod=OrdinalEncoder(categories=[nam])
df7.model=encod.fit_transform(df5[['model']])

In [ ]:
df7.vehicle_type.replace({'Car':0,'Passenger':1},inplace=True)

In [ ]:
name=df7.groupby('manufacturer')['price_in_thousands'].mean().sort_values().index
encod2=OrdinalEncoder(categories=[name])
df7.manufacturer=encod2.fit_transform(df5[['manufacturer']])

In [ ]:
plt.figure(figsize=(20,15))
mask = np.triu(np.ones_like(df7.corr(), dtype=bool))
heatmap(df7.corr(),annot=True,mask=mask)

there is strong multicollinearity between many of the features price of car 

vehicle_type and launch date has no relation with any of the values hence i will drop those columns

power_factor and horsepower has high collrellation hence we can drop horsepower column

year_resale_value , power_factor and model has highest influence on price of car

In [ ]:
drop_list=['wheelbase','length','width','sales_in_thousands',\
           'fuel_capacity','vehicle_type','days_after_launch(1/1/13)','horsepower']
df8=df7.drop(drop_list,axis=1).copy()

I will reduce all the column having correlation value less than 0.5 to 1 column using PCA

In [ ]:
pca=PCA(n_components=1)
df8['pca1']=pca.fit_transform(df7[['wheelbase','length','width','sales_in_thousands','fuel_capacity']])
df8.head(2)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
X=df8.drop('price_in_thousands',axis=1)

Scaling of data

In [ ]:
scale=StandardScaler()
X_scaled=scale.fit_transform(X)

In [ ]:
y=df8.price_in_thousands

### MODEL Building

In [ ]:
models=[Lasso(),Ridge(),LinearRegression(),RandomForestRegressor() \
         ,DecisionTreeRegressor(),SVR(C=10),KNeighborsRegressor()]

score=[]
for model in models:
    score.append(cross_val_score(model,X_scaled,y,n_jobs=-1).mean())
pd.DataFrame(score,index=models).sort_values(by=0,ascending=False)

Randomforestregressor ouperformed all other model with accuracy of 99%

## DATA PROCESSING AND BUILDING MODEL FOR TARGET AS SALES

###  Data Preprocessing

In [ ]:
df9=df5.copy()

In [ ]:
df9.drop(df9[df9.model=='Neon'].index,inplace=True)

I will encode categorical  columns with no according to their sales

In [ ]:
nam=df9[['model','sales_in_thousands']].sort_values(by='sales_in_thousands').model.values
encod=OrdinalEncoder(categories=[nam])
df9.model=encod.fit_transform(df9[['model']])

In [ ]:
df9.vehicle_type.replace({'Car':0,'Passenger':1},inplace=True)

In [ ]:
name=df9.groupby('manufacturer')['sales_in_thousands'].mean().sort_values().index
encod2=OrdinalEncoder(categories=[name])
df9.manufacturer=encod2.fit_transform(df9[['manufacturer']])

In [ ]:
plt.figure(figsize=(20,15))
mask = np.triu(np.ones_like(df9.corr(), dtype=bool))
heatmap(df9.corr(),annot=True,mask=mask)

most of the features donot have correlation with no of sales 

i will  use sklearn SelectKBest to find 5 most sutable column

In [ ]:
X = df9.drop('sales_in_thousands',axis=1)
y = df9.sales_in_thousands
bestfeatures = SelectKBest(score_func=mutual_info_regression,k=10)
fit = bestfeatures.fit(X,y)
dbest= pd.DataFrame(fit.scores_,columns=['score'])
dbest['col_nam']= pd.DataFrame(X.columns)
dbest.sort_values(by='score',ascending=False,inplace=True)
dbest

In [ ]:
most_related=dbest.col_nam.values[:5]
df10=df9[most_related].copy()

In [ ]:
pca=PCA(1)
df10['pca1']=pca.fit_transform(df9[dbest.col_nam.values[5:]])
df10.head(2)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
df10['sales']=df9.sales_in_thousands
plt.figure(figsize=(10,7))
mask = np.triu(np.ones_like(df10.corr(), dtype=bool))
heatmap(df10.corr(),annot=True,mask=mask)

In [ ]:
X_sales=df10.drop('sales',axis=1)
y_sales=df10.sales

In [ ]:
scale_sales=StandardScaler()
X_scaled_sales=scale.fit_transform(X)

### Model Building

In [ ]:
models_sales=[RandomForestRegressor(),DecisionTreeRegressor()]

score_sales=[]
for model in models_sales:
    score_sales.append(cross_val_score(model,X_scaled_sales,y_sales,n_jobs=-1).mean())
pd.DataFrame(score_sales,index=models_sales).sort_values(by=0,ascending=False)